In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math

In [2]:
data =  pd.read_csv('combo.gzip',compression='gzip',parse_dates=True,index_col='date')

In [3]:
data['departure_delay'] = data['departure_delay'].astype(int)
data['arrival_delay'] = data['arrival_delay'].astype(int)

In [4]:
data = data.reset_index()

In [5]:
data.head()

,date,airline,airline_code,departure_airport,departure_state,departure_lat,departure_lon,arrival_airport,arrival_state,arrival_lat,arrival_lon,departure_schedule,departure_actual,departure_delay,arrival_schedule,arrival_actual,arrival_delay
0,2012-11-25,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,1755.0,1805,10,1931.0,1948,17
1,2012-11-29,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,1755.0,1752,-3,1931.0,1929,-2
2,2012-11-29,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,1445.0,1442,-3,1623.0,1615,-8
3,2012-11-29,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,2005.0,2007,2,2140.0,2139,-1
4,2012-11-29,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,1630.0,1624,-6,1808.0,1805,-3


In [6]:
def makeDateTime(date,timeInInt):
    hour = math.floor(timeInInt//100)
    extraday = False
    if(hour==24):
        hour = 0
        extraday = True
        
    dateTime = datetime.datetime(date.year,date.month,date.day,hour,math.floor(timeInInt%100),0)
    
    if(extraday):
        dateTime = dateTime + datetime.timedelta(days = 1)
        
    return dateTime

In [7]:
def makeDateTimeWithDelay(dateTime, delayInMinutes):
    actual = dateTime
    delay = datetime.timedelta(minutes= math.floor(delayInMinutes))
    return actual-delay

In [8]:
data['departure_actual_datetime'] = np.nan
data['departure_schedule_datetime'] = np.nan
data['arrival_actual_datetime'] = np.nan
data['arrival_schedule_datetime'] = np.nan

In [9]:
def fillInRows(row):
    base_date = row['date']
    arrival_actual = row['arrival_actual']
    departure_actual = row['departure_actual']
    arrival_delay = row["arrival_delay"]
    departure_delay = row["departure_delay"]
    
    
    row["departure_actual_datetime"] = makeDateTime(base_date,departure_actual)
    row['arrival_actual_datetime'] = makeDateTime(base_date,arrival_actual)
    
    row['departure_schedule_datetime'] = makeDateTimeWithDelay(makeDateTime(base_date,departure_actual),departure_delay)
    row['arrival_schedule_datetime'] = makeDateTimeWithDelay(makeDateTime(base_date,arrival_actual),arrival_delay)
    return row


print(datetime.datetime.now())
data = data.apply(fillInRows,axis=1)
print(datetime.datetime.now())

2018-05-01 11:13:43.053207


KeyboardInterrupt: 

In [ ]:
data.tail()

In [245]:
data.to_csv('datetime.gzip',compression='gzip')